# Installing packages and start seetings, Data Loading ...

In [ ]:
!pip install -qU langchain_community faiss-cpu langchain_huggingface langchain_groq langgraph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 50.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from langchain_community.document_loaders import CSVLoader
DATA_PATH = "/content/drive/MyDrive/news_samples_for_rag_v2.csv"

def load_and_preprocess_data(file_path):
    # Load CSV file
    loader = CSVLoader(
        file_path=file_path,
        csv_args={
            "delimiter": ","
        },
        encoding = "utf_8",
        metadata_columns = ["news_agency_name","url","title","published_at","ner","topic","categories"]
    )
    data = loader.load()

    return data

data = load_and_preprocess_data(DATA_PATH)

# Setting Up the VectorDB and Imporing the Data

برای نگه داشتن بردارهای متون خبری، از پایگاه داده فایس استفاده شده که یک پکیج بسیار ساده برای پایتون است و به صورت امبدشده روی نوت‌بوک می‌توان از آن استفاده کرد

انتخاب‌های دیگری مانند ویوییت وجود دارد که در صورت لزوم می‌توان در پروژه‌های بزرگ‌تر از آن‌ها بهره برد

برای ساختن بردار‌های بازنمایی سعی کردم از مدلی متن‌باز استفاده کنم. احتمالا انتخاب اول بسیاری از افراد هم برای مدل زبانی بزرگ و هم بازنمایی، اوپن‌ای‌آی باشد اما در صورت جواب گرفتن، ترجیح من مدل‌های متن‌باز هستند

جستجو کردم و مدلی که در باکس زیر استفاده شده یک مدل آموزش‌دیده بر روی متون فارسی است که خوب کار می‌کند و وزن‌های آن روی هاگینگ‌فیس موجود است

یقینا با بهتر کردن مدل بازنمایی، مثلا با تغیر آن یا ریزتنظیم آن روی دادگان هدف تسک، کیفیت سیستم بالاتر می‌رود و این موضوع همیشه یک آپشن است

In [ ]:
import faiss
from uuid import uuid4
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings

EMBEDDING_MODEL_NAME = "heydariAI/persian-embeddings"

def setup_vector_store(documents):

    embeddings = HuggingFaceEmbeddings(
        model_name=EMBEDDING_MODEL_NAME,
        model_kwargs={"device": "cuda"}  # Change to "cuda" if GPU available otherwise use "cpu"
    )

    index = faiss.IndexFlatL2(len(embeddings.embed_query("life_web")))

    vectorstore = FAISS(
        embedding_function=embeddings,
        index=index,
        docstore=InMemoryDocstore(),
        index_to_docstore_id={},
    )
    uuids = [str(uuid4()) for _ in range(len(documents))]

    vectorstore.add_documents(documents=documents, ids=uuids)

    return vectorstore.as_retriever(search_type="mmr", search_kwargs={"k": 2})

In [ ]:
retriever = setup_vector_store(data)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.06k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.34k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/271 [00:00<?, ?B/s]

# Loading the Large Language Model

بازهم مانند مدل بازنمایی، در مورد مدل‌زبانی بزرگ هم ترجیح من استفاده از آپشن‌های متن‌باز بوده. مدل لاما از ورژن 3 به بعد روی متون فارسی طبق تجربه‌ام خوب جواب می‌دهد و همچنین جواب دادن به سوالات مربوط به اخبار فارسی تسک آن‌چنان سختی به نظر نمیاید که نیاز به مدل‌های بسیار بزرگ با منظق و قدرت تحلیل استثنایی وجود داشته باشد

مدل لاما از طریق گروک استفاده شده است که پرامپت فرستاده می‌شود و در فضای ابری پردازش می‌شود و پاسخ دریافت می‌شود

درصورت لزوم از این مدل‌های رایگان می‌توان به صورت لوکال هم از طریق اولاما یا ابزارهای مشابه بهره جست

یقینا درصورت جواب موردتایید نگرفتن، تغییر مدل زبانی بزرگ نیز همیشه یک آپشن است

In [ ]:
from langchain_groq import ChatGroq
import getpass

GROQ_MODEL = "llama-3.3-70b-versatile"
GROQ_API_KEY = getpass.getpass("GROQ_API_KEY: ")

def setup_groq_model(model_name, api_key=None):
    return ChatGroq(
        temperature=0.4,
        model_name=model_name,
        groq_api_key=api_key
    )

llm = setup_groq_model(GROQ_MODEL, GROQ_API_KEY)

# Simple RAG Using Pure LangChain

باکس زیر، یک سیستم رگ بسیار ساده با یک زنجیره لنگ‌چین است که بصورت پیشفرض همیشه برای سوال بازیابی اطلاعات را انجام می‌دهد و بر اساس فایل‌هایی که بازیابی شده، جواب را تولید می‌کند

سیستم فقط یکبار اجرا می‌شود و حافظه‌ای ندارد سازوکاری برای سوالاتی که اصلا برای آن‌ها نباید بازیابی اطلاعات انجام شود اندیشیده نشده است.
اگر به همین حد بسنده شود، می‌توان از زنجیره زیر بهره برد

در قسمت بعدی سیستم رگ پیچیده‌تر و کاراتری را پیاده کرده‌ام

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

def get_persian_prompt():
    template = """As a helpful AI assistant, you have to answer to the questions based on the provided context.
    Context is from Persian news.
    If you don't know the correct answer, respond with "من یک دستیار هوش مصنوعی هستم و بر اساس اطلاعاتی که در اختیار من گذاشته‌شده است، در حال حاضر جواب دقیقی برای پرسش شما ندارم." and don't hallucinate.
    Otherwise if user greets or speaks not about any specific matter which needs RAG, treat like a normal and polite AI chatbot and answer, Like greetings, thanking, excusing etc.

    Context:
    {context}

    Question:
    {question}

    Answer: """
    return ChatPromptTemplate.from_template(template)

def build_rag_chain(retriever, llm, prompt):
    return (
        {"context": retriever, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )

prompt = get_persian_prompt()
rag_chain = build_rag_chain(retriever, llm, prompt)

In [ ]:
response = rag_chain.invoke("قوه قضائیه در مورد فساد اقتصادی چه کارهایی انجام داده است؟")
print(response)

قوه قضائیه در مورد فساد اقتصادی اقداماتی انجام داده است. به عنوان مثال، در پرونده فساد پتروشیمی جم، قوه قضائیه با محکومیت متهمین و بازگرداندن بخشی از مبلغ فساد اقتصادی به صندوق دولت، اقداماتی را انجام داده است. همچنین، با پیگیری‌های دادگستری تهران و قوه قضائیه، تلاش‌ها برای بازگرداندن مبلغ باقی‌مانده ادامه دارد.


# Multiturn Conversational RAG system built using LangGraph

در این بخش با استفاده از لنگ‌گراف یک سیستم گفتگوی دارای حافظه که تصمیم میگیرد از بازیابی اطلاعات استفاده کند یا نکند پیاده کردم

در وهله اول مدل زبانی با توجه به پرامپت کاربر تصمیم میگیرد که آیا باید از بازیابی اطلاعات استفاده شود یا خیر. پیام های معمولی مثل سلام یا تشکر در همین مرحله فورا جواب داده می‌شوند. اگر قرار باشد از رگ استفاده شود، یک تابع از پایگاه داده اطلاعات را بازیابی می‌کند و دوباره در اختیار مدل زبانی قرار می‌دهد

مدل زبانی در مورد مسائل تخصصی که در پایگاه داده‌اش اطلاعاتی ندارد، نباید جوابی بدهد

وقتی جواب به کاربر نمایش داده شد، سیستم برای سوال بعدی از کاربر ورودی می گیرد و این چرخه دوباره تکرار می‌شود. در پاسخ سوال‌های بعدی، تاریخچه گفتگو در نظر گرفته می‌شود

In [ ]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph, END
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage, ChatMessage, FunctionMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# Define the function that calls the model
def first_llm_call(state: MessagesState):
    system_prompt = (
        "You are a helpful assistant which has to work in a multiturn RAG system."
        "You have to consider user prompt and decide if it is a specialized questoin which needs to enter RAG pipeline or not."
        "The data you have in disposal for RAG, contains Persian news articles"
        "If the question needs to invoke RAG system, answer just with 'rag'."
        "Otherwise If user prompts you with normal chat, like greeting or thanking or anything else, answer like a polite and normal chatbot."
        "consider chat history in order to answer, but always emphesize on the last user prompt in order to decide."
    )
    messages = [SystemMessage(content=system_prompt)] + state["messages"]
    response = llm.invoke(messages)
    return {"messages": response}

def planner(state: MessagesState):
  if "rag" in state["messages"][-1].content:
    return "rag"
  else:
    return "else"

def retrieve (state: MessagesState):
  retrieved_info = retriever.invoke(state["messages"][-2].content)[0].page_content
  return {"messages": FunctionMessage(content=retrieved_info.replace("content", "context 1") + retrieved_info.replace("content", "context 2") , name="retrieval")}

def rag_llm_call(state: MessagesState):
  prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content="""
              As a helpful AI assistant, you have to answer to the questions based on the provided context.
              Context is from Persian news articles.
              If you don't know the correct answer, respond with
              "من یک دستیار هوش مصنوعی هستم و بر اساس اطلاعاتی که در اختیار من گذاشته‌شده است، در حال حاضر جواب دقیقی برای پرسش شما ندارم."
              and don't hallucinate but try to not repeat yourself and generate innovative but correct and truthful answers.
              consider chat history in order to answer the user.
              """
              ),
        MessagesPlaceholder(variable_name="messages")
    ]
)
  chain = prompt | llm
  response = chain.invoke(
    {
        "messages": state["messages"]
    }
)
  return {"messages": response}

workflow = StateGraph(state_schema=MessagesState)
workflow.add_node("start", first_llm_call)
workflow.add_node("retrieve", retrieve)
workflow.add_node("rag", rag_llm_call)

workflow.add_edge(START, "start")
workflow.add_edge("retrieve", "rag")
workflow.add_edge("rag", END)

workflow.add_conditional_edges(
    "start",
    planner,
    {
        "rag": "retrieve",
        "else": END
    }
)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [ ]:
from IPython.display import display, HTML

greet_message = """
سلام. امیدوارم حالتان خوب باشد.<br>
من یک دستیار گفتگوی هوشمند هستم و با توجه به اطلاعاتی که در اختیارم گذاشته شده به سوالات شما پاسخ می‌دهم.<br>
چگونه می‌توان کمکتان کنم؟<br>
"""

def pretty_greet(greet_message):
    display(HTML(f"""
        <div dir="rtl" style="text-align:right; font-family:Tahoma; color:black;">
            {greet_message}
        </div>
    """))

def pretty_prompt():
    display(HTML("""
        <div dir="rtl" style="text-align:right; font-family:Tahoma; color:black;">
            🔷 سوال خود را بنویسید (یا 'خ' برای خروج):
        </div>
    """))

def pretty_print_user():
    display(HTML(f"""
        <div dir="rtl" style="text-align:right; font-family:Tahoma; color:blue;">
            👤:
        </div>
    """))

def pretty_print_bot(message):
    display(HTML(f"""
        <div dir="rtl" style="text-align:right; font-family:Tahoma; color:green;">
            🤖: {message}
        </div>
    """))

def pretty_print_end(message):
    display(HTML(f"""
        <div dir="rtl" style="text-align:right; font-family:Tahoma; color:red;">
            🔚 {message}
        </div>
    """))

In [ ]:
def start_chat(thread_id):
  pretty_greet(greet_message)
  while True:
      pretty_prompt()
      pretty_print_user()
      user_input = input()
      if user_input.strip() == "خ":
          pretty_print_end("گفت‌وگو پایان یافت.")
          break
      else:
          result = app.invoke(
            {"messages": [HumanMessage(content=str(user_input))]},
            config={"configurable": {"thread_id": str(thread_id)}},
            )
          # print(result)
          pretty_print_bot(result['messages'][-1].content)
          # pretty_print_bot("پیام دریافت شد.")
  for msg in result['messages']:
    print(msg)

In [ ]:
start_chat(thread_id=1)

سلام. خوبی؟


قوه قضائیه در مورد فساد مالی چه کارهایی کرده است؟


دیگر به جز اینها چه اقداماتی انجام شده. بیشتر میتونی بهم بگی؟


ممنون


در مورد استعفای وزیر کشور میانمار که جدیدا اتفاق افتاده به من توضیح بده.


خ


content='سلام. خوبی؟' additional_kwargs={} response_metadata={} id='24e03128-0d96-4a1b-bfb1-211e56132c8a'
content='سلام، ممنون. خوبم. شما چطورید؟' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 158, 'total_tokens': 172, 'completion_time': 0.085279468, 'prompt_time': 0.017589253, 'queue_time': 0.219455321, 'total_time': 0.102868721}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_2ddfbb0da0', 'finish_reason': 'stop', 'logprobs': None} id='run--3641b7b7-d592-45c2-af4d-55221e8a486b-0' usage_metadata={'input_tokens': 158, 'output_tokens': 14, 'total_tokens': 172}
content='قوه قضائیه در مورد فساد مالی چه کارهایی کرده است؟' additional_kwargs={} response_metadata={} id='551d3a3c-c7c2-4e9d-98c6-db6e0d5466f0'
content='rag' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 2, 'prompt_tokens': 197, 'total_tokens': 199, 'completion_time': 0.017031633, 'prompt_time': 0.012168042, 'queue_time': 0.055820

یقینا این سیستم بی‌نقص نیست و صرفا به عنوان یک دمو برای بررسی طراحی شده

مثلا اگر چت زیاد ادامه پیدا کند، کانتکست مدل زبانی تمام می‌شود یا سرعت پاسخگویی
رفته رفته کمتر می‌شود

در تاریخچه می‌توان گفتگوهای غیرضروری را بعد از چند مرحله حذف کرد یا مسائل کلیدی را خلاصه کرد تا طول پرامپت رفته رفته خیلی زیاد نشود

قسمت‌های مختلف سیستم را می‌توان بهینه‌تر کرد، مثلا تصمیم گیری اینکه باید از بازیابی اطلاعات استفاده شود یا نه را می‌توان توسط یک مدل کم‌هزینه رمزگذار ریزتنظیم‌شده انجام داد، زیرا یک تسک دسته‌بندی سرراست است و برای پرامپت‌های معمول و رندم مانند سلام یا تشکر، از پاسخ‌های ازپیش‌تعیین‌شده استفاده کرد و اصلا از مدل‌زبانی بزرگ استفاده نکرد

تعداد فایل خبری بازیابی شده یا چانک‌کردن یا این قبیل مسائل هم می‌توانند با
آزمون و خطلا بهینه‌تر شوند

مورد دیگر در انتقال پرامپت به سیستم بازیابی است. در این سیستم به صورت خیلی ساده، عین سوال کاربر را به سیستم بازیابی می‌دهیم و 2 فایل بسیار مشابه را بازیابی می‌کنیم. میتوان به سیستم بازیابی به عنوان یک ابزار نگاه کرد که ورودی آن توسط مدل‌زبانی یک بار بازنویسی می‌شود و مثلا از حالت سوالی خارج می‌شود تا بهتر در سیستم بازیابی مشابهت‌یابی شود

به موارد این چنینی بسیار زیادی می‌توان فکر کرد